In [219]:
import feedparser
from bs4 import BeautifulSoup
import urllib
import xmltodict
import pandas as pd
import re

# build list of links to parse from:

def build_company_list():
    startnum = 0
    entryamt = 100
    url='https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent&CIK=&type=13f-hr&company=&dateb=&owner=include&start={}&count={}&output=atom'.format(startnum, entryamt)
    d = feedparser.parse(url)
    entrylist = d['entries']
    entries = d['entries']

    while len(entries) == 100:
        url='https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent&CIK=&type=13f-hr&company=&dateb=&owner=include&start={}&count={}&output=atom'.format(startnum, entryamt)
        d = feedparser.parse(url)
        entries = d['entries']
        entrylist = entrylist + d['entries']
        startnum += 100
    return entrylist
    

companylist = build_company_list()
print(len(companylist))


518


In [217]:
def parse_link(entrylist, list_index):
    url = entrylist[list_index]['link']
    print(list_index, url)
    page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')

    # get company name and CIK
    companyname = soup.find('span', class_='companyName').contents[0]
    contents = soup.find('span', class_='companyName').contents[3]
    companyCIK = re.sub('<.*?>', '', str(contents))
    companyCIK = re.sub(' \(.*?\)', '', companyCIK)

    #find report date and filing date
    links = soup.find_all('div', class_='formGrouping')
    filingDate = links[0].find_all('div', class_='info')[0].contents[0]
    reportDate = links[1].find_all('div', class_='info')[0].contents[0]

    # find the securities
    for link in soup.find_all('a'):
        securities_url = link.get('href')
        text = link.contents[0]
        if '.xml' in securities_url and 'xml' in text and 'primary_doc' not in securities_url:
            securities_list = securities_url

    # build proper url
    securities_list_url = 'https://www.sec.gov{}'.format(securities_list)
    data = urllib.request.urlopen(securities_list_url).read()
    namespaces = {'http://www.sec.gov/edgar/document/thirteenf/informationtable': None}
    securities = xmltodict.parse(data, process_namespaces=True, namespaces=namespaces)
    securities_list = securities['informationTable']['infoTable']

    if isinstance(securities_list,list):
        df = pd.DataFrame.from_dict(securities_list)
    else:
        df = pd.DataFrame.from_dict(securities_list, orient='index').transpose()
        
    if 'putCall' not in df.columns:
        df.insert(5, 'putCall', '')
    if 'otherManager' not in df.columns:
        df.insert(7, 'otherManager', '')
        
    df['sshPrnamt'] = [d.get('sshPrnamt') for d in df.shrsOrPrnAmt]
    df['sshPrnamtType'] = [d.get('sshPrnamtType') for d in df.shrsOrPrnAmt]
    df['votingSole'] = [d.get('Sole') for d in df.votingAuthority]
    df['votingShared'] = [d.get('Shared') for d in df.votingAuthority]
    df['votingNone'] = [d.get('Shared') for d in df.votingAuthority]
    df['companyName'] = companyname
    df['companyCIK'] = companyCIK
    df['reportDate'] = reportDate
    df['effectiveDate'] = effectiveDate
    return df




In [218]:
#go through each and every company item and then dump into csv file
for i in range(len(companylist)):
    df = parse_link(companylist, i)
    df.drop(columns=['shrsOrPrnAmt', 'votingAuthority'], inplace=True)
    
    if os.path.isfile('test.csv'):
        pass
    else:
        pd.DataFrame(df.columns).transpose().to_csv('test.csv', header=False, index=False, mode='a', sep='^')

    df.to_csv('test.csv', header=False, index=False, mode='a', sep='^')

0 https://www.sec.gov/Archives/edgar/data/1336098/000095012319004056/0000950123-19-004056-index.htm
1 https://www.sec.gov/Archives/edgar/data/752365/000075236519000002/0000752365-19-000002-index.htm
2 https://www.sec.gov/Archives/edgar/data/1280051/000128005119000004/0001280051-19-000004-index.htm
3 https://www.sec.gov/Archives/edgar/data/1042063/000104206319000004/0001042063-19-000004-index.htm
4 https://www.sec.gov/Archives/edgar/data/1512073/000151207319000005/0001512073-19-000005-index.htm
5 https://www.sec.gov/Archives/edgar/data/1479983/000114420419024414/0001144204-19-024414-index.htm
6 https://www.sec.gov/Archives/edgar/data/810265/000081026519000003/0000810265-19-000003-index.htm
7 https://www.sec.gov/Archives/edgar/data/1219037/000108514619001406/0001085146-19-001406-index.htm
8 https://www.sec.gov/Archives/edgar/data/1626982/000162698219000003/0001626982-19-000003-index.htm
9 https://www.sec.gov/Archives/edgar/data/1595509/000139834419008152/0001398344-19-008152-index.htm
10

82 https://www.sec.gov/Archives/edgar/data/1486713/000114420419024312/0001144204-19-024312-index.htm
83 https://www.sec.gov/Archives/edgar/data/1730610/000114420419024310/0001144204-19-024310-index.htm
84 https://www.sec.gov/Archives/edgar/data/1251557/000156761919010051/0001567619-19-010051-index.htm
85 https://www.sec.gov/Archives/edgar/data/1631507/000163150719000002/0001631507-19-000002-index.htm
86 https://www.sec.gov/Archives/edgar/data/1291424/000117266119001115/0001172661-19-001115-index.htm
87 https://www.sec.gov/Archives/edgar/data/1107314/000110731419000004/0001107314-19-000004-index.htm
88 https://www.sec.gov/Archives/edgar/data/1091961/000117266119001114/0001172661-19-001114-index.htm
89 https://www.sec.gov/Archives/edgar/data/1629283/000114420419024286/0001144204-19-024286-index.htm
90 https://www.sec.gov/Archives/edgar/data/1445891/000106299319002038/0001062993-19-002038-index.htm
91 https://www.sec.gov/Archives/edgar/data/1175399/000108514619001390/0001085146-19-001390-

163 https://www.sec.gov/Archives/edgar/data/1580677/000108514619001395/0001085146-19-001395-index.htm
164 https://www.sec.gov/Archives/edgar/data/1391974/000108514619001394/0001085146-19-001394-index.htm
165 https://www.sec.gov/Archives/edgar/data/1600052/000108514619001393/0001085146-19-001393-index.htm
166 https://www.sec.gov/Archives/edgar/data/1653253/000109544919000025/0001095449-19-000025-index.htm
167 https://www.sec.gov/Archives/edgar/data/1657134/000165713419000004/0001657134-19-000004-index.htm
168 https://www.sec.gov/Archives/edgar/data/1546587/000108514619001392/0001085146-19-001392-index.htm
169 https://www.sec.gov/Archives/edgar/data/1724090/000172409019000002/0001724090-19-000002-index.htm
170 https://www.sec.gov/Archives/edgar/data/1722638/000172263819000004/0001722638-19-000004-index.htm
171 https://www.sec.gov/Archives/edgar/data/1540945/000108514619001391/0001085146-19-001391-index.htm
172 https://www.sec.gov/Archives/edgar/data/1729673/000172967319000002/0001729673-

244 https://www.sec.gov/Archives/edgar/data/1775361/000177536119000003/0001775361-19-000003-index.htm
245 https://www.sec.gov/Archives/edgar/data/1645173/000161577419007182/0001615774-19-007182-index.htm
246 https://www.sec.gov/Archives/edgar/data/1234074/000095012319003984/0000950123-19-003984-index.htm
247 https://www.sec.gov/Archives/edgar/data/1259261/000125926119000004/0001259261-19-000004-index.htm
248 https://www.sec.gov/Archives/edgar/data/1357867/000135786719000007/0001357867-19-000007-index.htm
249 https://www.sec.gov/Archives/edgar/data/1715862/000171586219000003/0001715862-19-000003-index.htm
250 https://www.sec.gov/Archives/edgar/data/1715635/000171563519000002/0001715635-19-000002-index.htm
251 https://www.sec.gov/Archives/edgar/data/1097833/000109783319000004/0001097833-19-000004-index.htm
252 https://www.sec.gov/Archives/edgar/data/1775361/000177536119000002/0001775361-19-000002-index.htm
253 https://www.sec.gov/Archives/edgar/data/1602905/000173081019000002/0001730810-

325 https://www.sec.gov/Archives/edgar/data/1313473/000131347319000011/0001313473-19-000011-index.htm
326 https://www.sec.gov/Archives/edgar/data/1707202/000170720619000004/0001707206-19-000004-index.htm
327 https://www.sec.gov/Archives/edgar/data/1564835/000121390019007935/0001213900-19-007935-index.htm
328 https://www.sec.gov/Archives/edgar/data/1728321/000139834419008073/0001398344-19-008073-index.htm
329 https://www.sec.gov/Archives/edgar/data/1509974/000139834419008071/0001398344-19-008071-index.htm
330 https://www.sec.gov/Archives/edgar/data/1707206/000170720619000003/0001707206-19-000003-index.htm
331 https://www.sec.gov/Archives/edgar/data/902528/000117891319001365/0001178913-19-001365-index.htm
332 https://www.sec.gov/Archives/edgar/data/1219573/000095012319003959/0000950123-19-003959-index.htm
333 https://www.sec.gov/Archives/edgar/data/1411530/000095012319003956/0000950123-19-003956-index.htm
334 https://www.sec.gov/Archives/edgar/data/1750451/000175045119000003/0001750451-1

406 https://www.sec.gov/Archives/edgar/data/1771687/000108514619001369/0001085146-19-001369-index.htm
407 https://www.sec.gov/Archives/edgar/data/1764403/000176440319000005/0001764403-19-000005-index.htm
408 https://www.sec.gov/Archives/edgar/data/1733777/000108514619001367/0001085146-19-001367-index.htm
409 https://www.sec.gov/Archives/edgar/data/884541/000108514619001366/0001085146-19-001366-index.htm
410 https://www.sec.gov/Archives/edgar/data/1730456/000108514619001365/0001085146-19-001365-index.htm
411 https://www.sec.gov/Archives/edgar/data/1122490/000108514619001364/0001085146-19-001364-index.htm
412 https://www.sec.gov/Archives/edgar/data/1569356/000108514619001363/0001085146-19-001363-index.htm
413 https://www.sec.gov/Archives/edgar/data/1266014/000126601419000008/0001266014-19-000008-index.htm
414 https://www.sec.gov/Archives/edgar/data/1454949/000108514619001362/0001085146-19-001362-index.htm
415 https://www.sec.gov/Archives/edgar/data/1737112/000173711219000003/0001737112-1

In [216]:

print(reportDate)

2019-03-31
